In [438]:
import pandas as pd
import numpy as np

import pandascharm as pc
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62 as blosum
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
from Bio.Align import AlignInfo

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings("ignore")

In [439]:
df = pd.read_excel('Review Paper Data Final.xlsx')

In [440]:
# Download
# pip install pandas-charm

In [441]:
sequence=df["Sequence"]
organisms=df["Organism Name"]

count=0
ofile = open("seqs.fasta", "w")
for i in range(len(sequence)):
    if str(sequence[i]) != 'nan':
        ofile.write(">" + str(organisms[i]) + "\n" + str(sequence[i]) + "\n")
        count=count+1
ofile.close()

In [442]:
from Bio.Align.Applications import ClustalwCommandline
in_file="seqs.fasta"
clustalw_cline=ClustalwCommandline("clustalw2", infile=in_file)

print(clustalw_cline)

clustalw2 -infile=seqs.fasta


In [374]:
# import os
# clustalw_exe =  r"/Users/ryanjang123/Documents/clustalw/clustalw2.exe"
# clustalw_cline=ClustalwCommandline(clustalw_exe, infile='seqs.fasta')

In [443]:
from Bio import AlignIO
ClustalAlign = AlignIO.read('seqs.aln', 'clustal')
summary_align = AlignInfo.SummaryInfo(ClustalAlign )

In [498]:
dframe = pc.from_bioalignment(ClustalAlign)

In [499]:
dframe

,E6,E12,E9,E10,E7,E8,E11,E307,E308,E309,...,E278,E273,E64,E70,E63,E62,E66,E261,E65,E310
0,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294,-,-,-,-,-,-,-,-,-,-,...,-,-,E,-,-,-,-,-,-,-
1295,-,-,-,-,-,-,-,-,-,-,...,-,-,K,-,-,-,-,-,-,-
1296,-,-,-,-,-,-,-,-,-,-,...,-,-,A,-,-,-,-,-,-,-
1297,-,-,-,-,-,-,-,-,-,-,...,-,-,R,-,-,-,-,-,-,-


In [500]:
dframe['E6']

0       -
1       -
2       -
3       -
4       -
       ..
1294    -
1295    -
1296    -
1297    -
1298    -
Name: E6, Length: 1299, dtype: category
Categories (21, object): ['-', 'A', 'C', 'D', ..., 'T', 'V', 'W', 'Y']

In [501]:
# changing U or O to -
dframe = dframe.replace('U', '-')
dframe = dframe.replace('O','-')

In [502]:
data=df[['Wild/Mutant', 'Kingdom', 'pH Optimum', 'Temperature Optimum',
       't1/2 (min)', 'kd (min-1)', 'pNP-Glc Km (mM)', 'pNP-Glc kcat (1/s)',
       'pNP-Glc kcat/Km (1/smM)', 'Cellobiose Km (mM)',
       'Cellobiose kcat (1/s)', 'Cellobiose kcat/Km (1/smM)', 'Cellobiose Ki (mM)',
       'pNP-Glc Ki (mM)', 'MW (kDa)']]

In [503]:
general_info=data[['Wild/Mutant','Kingdom','pH Optimum','Temperature Optimum','MW (kDa)']]
stability_df=data[['t1/2 (min)','kd (min-1)']]
kinetics_pnp=data[['pNP-Glc Km (mM)', 'pNP-Glc kcat (1/s)', 'pNP-Glc kcat/Km (1/smM)', 'pNP-Glc Ki (mM)']]
kinetics_cellobiose=data[['Kingdom','Cellobiose Km (mM)', 'Cellobiose kcat (1/s)', 'Cellobiose kcat/Km (1/smM)', 'Cellobiose Ki (mM)']]


#removal of outliers

def removeoutliers(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df_out.dropna()

In [504]:
kcatPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc kcat (1/s)']])
kcatPnpGlc

,pNP-Glc kcat (1/s)
0,26.640000
2,5.500000
3,1.700000
4,169.000000
5,277.000000
...,...
306,19.848533
307,14.464000
308,35.558400
309,20.250000


In [505]:
# Removing Outliers
kcatPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc kcat (1/s)']])
KmPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc Km (mM)']])
kcatKmPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc kcat/Km (1/smM)']])
KiPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc Ki (mM)']])
stability=removeoutliers(stability_df[['t1/2 (min)']])

kcat_index = list(kcatPnpGlc.index)
Km_index = list(KmPnpGlc.index)
kcatKm_index = list(kcatKmPnpGlc.index)
Ki_index = list(KiPnpGlc.index)
stability_index = list(stability.index)

# kcatKmCellobiose=kinetics_cellobiose[['Cellobiose kcat/Km (1/smM)']]
# kcatCellobiose=kinetics_cellobiose[[ 'Cellobiose kcat (1/s)']]
# KmCellobiose=kinetics_cellobiose[['Cellobiose Km (mM)']]

In [506]:
# Machine Learning Tools

def LR(X_train, y_train, X_test):
    from sklearn.linear_model import LinearRegression
    lm = LinearRegression()
    lm.fit(X_train,y_train)
    lin_pred = lm.predict(X_test)
    pred_training= lm.predict(X_train)
    
    return lin_pred , pred_training

#Ordinary Least Squares Regression
def OLS(X_train, y_train, X_test):
    import statsmodels.api as sm
    from scipy import stats
    X2 = sm.add_constant(X_train)
    est = sm.OLS(y_train,X2)
    est2 = est.fit()
    print(est2.summary())
    coefficients=est2.params[1:]
    intercept=est2.params[0]
    OLS_pred=(coefficients*X_test).sum(axis = 1, skipna = True) + intercept
    
    return OLS_pred

#Lasso regression

def LASSO(X_train, y_train, X_test):
    param_grid={'alpha':np.arange(0,1.1,0.05).tolist()}
    lasso_reg=Lasso()
    grid_search=GridSearchCV(lasso_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    lasso_pred=grid_search.predict(X_test)
    pred_training= grid_search.predict(X_train)
    return lasso_pred, pred_training

#Partial Least Square Regression

def PLS(X_train, y_train, X_test):
    param_grid={'n_components':np.arange(1,10,1).tolist()}
    pls = PLSRegression()
    grid_search=GridSearchCV(pls, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    pls.fit(scale(X_train), y_train)
    PLS_pred=pls.predict(scale(X_test))
    pred_training= pls.predict(X_train)
    
    return PLS_pred , pred_training

#Random Forest

def RF(X_train, y_train, X_test):
    param_grid={"n_estimators": [100] , "max_features": [5], 'random_state':[20]}
    rf = RandomForestRegressor()
    grid_search=GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    RF_pred = grid_search.predict(X_test)
    pred_training= grid_search.predict(X_train)
    return RF_pred, pred_training

#Decision Tree

def DT(X_train, y_train, X_test):
    tree_reg=DecisionTreeRegressor()
    tree_reg.fit(X_train, y_train)
    DT_pred=tree_reg.predict(X_test)
    pred_training= tree_reg.predict(X_train)
    return DT_pred, pred_training

#SVM

def SVM(X_train, y_train, X_test):
    model = SVR(kernel='poly', degree=2, C=100, epsilon=0.5)
    model.fit(X_train,y_train)
    SVM_pred = model.predict(X_test)
    pred_training= model.predict(X_train)
    return SVM_pred, pred_training

#Neural Network Regression

def NNR(X_train, y_train, X_test):
    regr = MLPRegressor(random_state=101, max_iter=100).fit(X_train, y_train)
    NNR_pred=regr.predict(X_test)
    pred_training= regr.predict(X_train)
    return NNR_pred, pred_training

#Elastic Net

def EN(X_train, y_train, X_test):
    ENmodel = ElasticNet(alpha=1.0, l1_ratio=0.5)
    ENmodel.fit(X_train,y_train)
    EN_pred = ENmodel.predict(X_test)
    pred_training= ENmodel.predict(X_train)
    return EN_pred, pred_training

In [507]:
# 1 pNp OneHot Encoder
X = df['Sequence'].iloc[kcat_index].dropna()
rem_kcat_index = list(X.index)
y = kinetics_pnp['pNP-Glc kcat (1/s)'].iloc[rem_kcat_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()

In [508]:
encoded.transform(temp_seq).toarray().shape

(151, 146)

In [456]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [457]:
lin_pred, lin_train = LR(X_train, y_train, X_test)
lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
DT_pred , DT_train = DT(X_train, y_train, X_test)
RF_pred , RF_train = RF(X_train, y_train, X_test)
SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
EN_pred , EN_train = EN(X_train, y_train, X_test)

In [458]:
results=[]
results.append(["Linear Regression",  np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
results.append(["LASSO Regression",   np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
results.append(["Partial Least Square",np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
results.append(["Decision Tree Regression",  np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
results.append(["Random Forest Regression",np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
results.append(["Support Vector Machine Regression", np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
results.append(["Neural Network Regression", np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
results.append(["Elastic Network Regression", np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 

In [459]:
results

[['Linear Regression',
  20.610300698353317,
  46.204811787408175,
  0.9478425824094823,
  -0.048261373422832765],
 ['LASSO Regression',
  59.635533766228676,
  48.66951159510327,
  0.5633256867410035,
  -0.16307882279155916],
 ['Partial Least Square',
  82.33500223942136,
  55.027097194281225,
  0.16762998351935476,
  -0.48678571192596554],
 ['Decision Tree Regression',
  20.610300698353317,
  49.0719724312433,
  0.9478425824094823,
  -0.1823939568137205],
 ['Random Forest Regression',
  39.31473145789976,
  45.72930709951454,
  0.8102166748713724,
  -0.026796580994491226],
 ['Support Vector Machine Regression',
  54.53179373179447,
  47.099873092540946,
  0.6348704423138152,
  -0.08926775086847538],
 ['Neural Network Regression',
  102.2839668913946,
  48.46928863588343,
  -0.2845841241555367,
  -0.1535288577229501],
 ['Elastic Network Regression',
  89.37299355262758,
  54.85472028453123,
  0.019246121636264713,
  -0.4774853447015628]]

In [460]:
# 2 KmpNp OneHot Encoder
X = df['Sequence'].iloc[Km_index].dropna()
rem_km_index = list(X.index)
y = kinetics_pnp['pNP-Glc Km (mM)'].iloc[rem_km_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()

In [461]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [462]:
lin_pred, lin_train = LR(X_train, y_train, X_test)
lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
DT_pred , DT_train = DT(X_train, y_train, X_test)
RF_pred , RF_train = RF(X_train, y_train, X_test)
SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
EN_pred , EN_train = EN(X_train, y_train, X_test)

In [463]:
results=[]
results.append(["Linear Regression",  np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
results.append(["LASSO Regression",   np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
results.append(["Partial Least Square",np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
results.append(["Decision Tree Regression",  np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
results.append(["Random Forest Regression",np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
results.append(["Support Vector Machine Regression", np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
results.append(["Neural Network Regression", np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
results.append(["Elastic Network Regression", np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 

In [464]:
results

[['Linear Regression',
  0.032064161983620734,
  805828372196.6648,
  0.9999000080580308,
  -6.308369958693032e+22],
 ['LASSO Regression',
  2.991812922989509,
  3.2400948820786617,
  0.1294492724182127,
  -0.019876306900053153],
 ['Partial Least Square',
  2.9210954028779255,
  3.104706806816615,
  0.17011732567960225,
  0.0635744947482082],
 ['Decision Tree Regression',
  0.03203031897020904,
  4.516724563069492,
  0.9999002190252003,
  -0.9818886985419957],
 ['Random Forest Regression',
  1.2484150678533075,
  3.438718065644696,
  0.8484195406344782,
  -0.14874910715393574],
 ['Support Vector Machine Regression',
  0.4779446866335807,
  3.048905681508813,
  0.9777832518926991,
  0.09693288979073045],
 ['Neural Network Regression',
  3.1159135570126857,
  3.9036327558804693,
  0.055730381020835495,
  -0.48036890249408093],
 ['Elastic Network Regression',
  3.206545393172229,
  3.2486320654555407,
  0.0,
  -0.025257841729539265]]

In [465]:
# 3 kcat/Km OneHot Encoder
X = df['Sequence'].iloc[kcatKm_index].dropna()
rem_index = list(X.index)
y = kinetics_pnp['pNP-Glc kcat/Km (1/smM)'].iloc[rem_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()

In [466]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [467]:
lin_pred, lin_train = LR(X_train, y_train, X_test)
lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
DT_pred , DT_train = DT(X_train, y_train, X_test)
RF_pred , RF_train = RF(X_train, y_train, X_test)
SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
EN_pred , EN_train = EN(X_train, y_train, X_test)

In [468]:
results=[]
results.append(["Linear Regression",  np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
results.append(["LASSO Regression",   np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
results.append(["Partial Least Square",np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
results.append(["Decision Tree Regression",  np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
results.append(["Random Forest Regression",np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
results.append(["Support Vector Machine Regression", np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
results.append(["Neural Network Regression", np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
results.append(["Elastic Network Regression", np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 

In [469]:
results

[['Linear Regression',
  1.1314996437964915,
  190088305250160.3,
  0.9999509061564746,
  -2.5517901561806183e+24],
 ['LASSO Regression',
  1.102126379812885,
  119.6730281874977,
  0.9999534219839757,
  -0.01140878445364768],
 ['Partial Least Square',
  146.77963666163708,
  120.2071608101485,
  0.17386730990794907,
  -0.020457306448618873],
 ['Decision Tree Regression',
  1.1021245670538662,
  129.76225864225208,
  0.9999534221371971,
  -0.1891344275004656],
 ['Random Forest Regression',
  62.58948928738388,
  120.02814859154645,
  0.8497823351242626,
  -0.017420244328540502],
 ['Support Vector Machine Regression',
  132.24673752567878,
  120.82682766367272,
  0.3293620511554596,
  -0.03100532083857721],
 ['Neural Network Regression',
  174.72167292849272,
  129.02668866414723,
  -0.17060870569665898,
  -0.1756911897395761],
 ['Elastic Network Regression',
  159.4065488580269,
  120.30300981667757,
  0.025615179216125772,
  -0.02208530952999621]]

In [470]:
# 4 Ki
X = df['Sequence'].iloc[Ki_index].dropna()
rem_index = list(X.index)
y = kinetics_pnp['pNP-Glc Ki (mM)'].iloc[rem_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()

In [471]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [472]:
lin_pred, lin_train = LR(X_train, y_train, X_test)
lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
DT_pred , DT_train = DT(X_train, y_train, X_test)
RF_pred , RF_train = RF(X_train, y_train, X_test)
SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
EN_pred , EN_train = EN(X_train, y_train, X_test)

In [473]:
results=[]
results.append(["Linear Regression",  np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
results.append(["LASSO Regression",   np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
results.append(["Partial Least Square",np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
results.append(["Decision Tree Regression",  np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
results.append(["Random Forest Regression",np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
results.append(["Support Vector Machine Regression", np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
results.append(["Neural Network Regression", np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
results.append(["Elastic Network Regression", np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 

In [474]:
results

[['Linear Regression',
  2.5109329991367936e-13,
  415.4917289558565,
  1.0,
  -0.4607868569161404],
 ['LASSO Regression',
  0.0019510530464765723,
  411.2340769013611,
  0.9999999999346724,
  -0.4310021213901405],
 ['Partial Least Square',
  201.7215945926389,
  411.2711347885382,
  0.30166492622158336,
  -0.4312600392403789],
 ['Decision Tree Regression',
  0.0,
  564.0875301759471,
  1.0,
  -1.6924961160375922],
 ['Random Forest Regression',
  89.15494314926313,
  414.16111715441934,
  0.8635888328987564,
  -0.45144550216471924],
 ['Support Vector Machine Regression',
  197.56920663997653,
  465.58571059986576,
  0.3301191208200869,
  -0.834262015520949],
 ['Neural Network Regression',
  318.9986966287697,
  564.6345552815561,
  -0.7463725125171743,
  -1.697720756007202],
 ['Elastic Network Regression',
  229.3829910774763,
  418.20559149590537,
  0.0970130106818271,
  -0.47993198777280055]]

In [475]:
# 5 stability
X = df['Sequence'].iloc[stability_index].dropna()
rem_index = list(X.index)
y = stability_df['t1/2 (min)'].iloc[rem_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()

In [476]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [477]:
lin_pred, lin_train = LR(X_train, y_train, X_test)
lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
DT_pred , DT_train = DT(X_train, y_train, X_test)
RF_pred , RF_train = RF(X_train, y_train, X_test)
SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
EN_pred , EN_train = EN(X_train, y_train, X_test)

In [478]:
results=[]
results.append(["Linear Regression",  np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
results.append(["LASSO Regression",   np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
results.append(["Partial Least Square",np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
results.append(["Decision Tree Regression",  np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
results.append(["Random Forest Regression",np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
results.append(["Support Vector Machine Regression", np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
results.append(["Neural Network Regression", np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
results.append(["Elastic Network Regression", np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 

In [479]:
results

[['Linear Regression',
  103.86535755486524,
  262.63320456100945,
  0.8334903943840593,
  -0.07231225572191557],
 ['LASSO Regression',
  103.86535755486524,
  258.7508335530364,
  0.8334903943840593,
  -0.04084370293208739],
 ['Partial Least Square',
  220.9022014052154,
  257.902013319718,
  0.24682145730991745,
  -0.034026023856178256],
 ['Decision Tree Regression',
  103.86535755486524,
  307.8598226776241,
  0.8334903943840593,
  -0.4734249981277716],
 ['Random Forest Regression',
  129.41587210587141,
  274.41213805551666,
  0.7414926249378673,
  -0.17065423126577794],
 ['Support Vector Machine Regression',
  230.37378639221055,
  288.79351099229234,
  0.18084898839619323,
  -0.29657267711558055],
 ['Neural Network Regression',
  309.5415034940825,
  338.46216337224746,
  -0.47888916316224517,
  -0.7809111628135621],
 ['Elastic Network Regression',
  244.3947261633752,
  258.1714756347473,
  0.07810489057881198,
  -0.036187903888502015]]

In [480]:
# Outliers comparison dataframe

AA1=kinetics_pnp['pNP-Glc kcat (1/s)'].dropna()
AA1_ind = list(AA1.index)
A1 = df['Sequence'].iloc[AA1_ind].dropna()


AA2=kinetics_pnp['pNP-Glc Km (mM)'].dropna()
AA2_ind = list(AA2.index)
A2 = df['Sequence'].iloc[AA2_ind].dropna()

AA3=kinetics_pnp['pNP-Glc kcat/Km (1/smM)'].dropna()
AA3_ind = list(AA3.index)
A3 = df['Sequence'].iloc[AA3_ind].dropna()


AA4=kinetics_pnp['pNP-Glc Ki (mM)'].dropna()
AA4_ind = list(AA4.index)
A4 = df['Sequence'].iloc[AA4_ind].dropna()

AA5=stability_df['t1/2 (min)'].dropna()
AA5_ind = list(AA5.index)
A5 = df['Sequence'].iloc[AA5_ind].dropna()

BB1 = removeoutliers(kinetics_pnp[['pNP-Glc kcat (1/s)']])
BB1_ind = list(BB1.index)
B1 = df['Sequence'].iloc[BB1_ind].dropna()

BB2 = removeoutliers(kinetics_pnp[['pNP-Glc Km (mM)']])
BB2_ind = list(BB2.index)
B2 = df['Sequence'].iloc[BB2_ind].dropna()

BB3 = removeoutliers(kinetics_pnp[['pNP-Glc kcat/Km (1/smM)']])
BB3_ind = list(BB3.index)
B3 = df['Sequence'].iloc[BB3_ind].dropna()

BB4= removeoutliers(kinetics_pnp[['pNP-Glc Ki (mM)']])
BB4_ind = list(BB4.index)
B4 = df['Sequence'].iloc[BB4_ind].dropna()

BB5 = removeoutliers(stability_df[['t1/2 (min)']])
BB5_ind = list(BB5.index)
B5 = df['Sequence'].iloc[BB5_ind].dropna()

In [481]:
frame = pd.DataFrame( columns=['w/outliers','w/o outliers' ])
dict1 = pd.Series({'w/outliers':len(A1) ,'w/o outliers':len(B1)}, name = 'pNP-Glc kcat (1/s)')
dict2 = pd.Series({'w/outliers':len(A2),'w/o outliers':len(B2)}, name = 'pNP-Glc Km (mM)')
dict3 = pd.Series({'w/outliers':len(A3),'w/o outliers':len(B3)},name = 'pNP-Glc kcat/Km (1/smM)')
dict4 = pd.Series({'w/outliers':len(A4),'w/o outliers':len(B4)}, name = 'pNP-Glc Ki (mM)')
dict5 = pd.Series({'w/outliers':len(A5),'w/o outliers':len(B5)}, name = 't1/2 (min)')

In [482]:
frame = frame.append(dict1)
frame = frame.append(dict2)
frame = frame.append(dict3)
frame =frame.append(dict4)
frame = frame.append(dict5)

In [483]:
frame

,w/outliers,w/o outliers
pNP-Glc kcat (1/s),174,151
pNP-Glc Km (mM),188,168
pNP-Glc kcat/Km (1/smM),175,151
pNP-Glc Ki (mM),56,45
t1/2 (min),62,50


In [509]:
#AAIndex
from pyaaisc import Aaindex
aaindex = Aaindex()
full_list = aaindex.get_all(dbkey='aaindex1')

In [510]:
record = aaindex.get(full_list[0][0])
record.title

'Peptide/protein structure analysis using the chemical shift index method: upfield alpha-CH values reveal dynamic helices and aL sites'

In [511]:
c = record.index_data

In [512]:
c['-'] = 0

In [513]:
sequence_aligned=dframe.apply(lambda dframe : pd.Series(c[i] for i in dframe))

In [514]:
# org=list(dframe.columns)
# sequence_aligned.set_axis(org, axis=1, inplace=True)

In [519]:
sequence_aligned['E6'].sum()

2007.02

In [516]:
X = sequence_aligned.transpose()

In [517]:
X.iloc[0]

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1294    0.0
1295    0.0
1296    0.0
1297    0.0
1298    0.0
Name: E6, Length: 1299, dtype: float64

In [524]:
# 1. kcat
X = sequence_aligned.iloc[kcat_index].dropna()
rem_kcat_index = list(X.index)
y = kinetics_pnp['pNP-Glc kcat (1/s)'].iloc[rem_kcat_index]



In [ ]:
for key in range(len(full_list)):
    try:
        record = aaindex.get(full_list[key][0])
        title = record.title
        index_data = record.index_data
        index_data['-'] = 0
        sequence_aligned=dframe.apply(lambda dframe : pd.Series(index_data[i] for i in dframe))

#         org=list(dframe.index)
        X = sequence_aligned.iloc[kcat_index].dropna()
        rem_kcat_index = list(X.index)
        y = kinetics_pnp['pNP-Glc kcat (1/s)'].iloc[rem_kcat_index]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)
        
        lin_pred, lin_train = LR(X_train, y_train, X_test)
        lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
        PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
        DT_pred , DT_train = DT(X_train, y_train, X_test)
        RF_pred , RF_train = RF(X_train, y_train, X_test)
        SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
        NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
        EN_pred , EN_train = EN(X_train, y_train, X_test)


        # Add rows 
        results.append(["Linear Regression", full_list[key][0], title, np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
        results.append(["LASSO Regression",  full_list[key][0], title, np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
        results.append(["Partial Least Square", full_list[key][0], title,np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
        results.append(["Decision Tree Regression", full_list[key][0], title, np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
        results.append(["Random Forest Regression",full_list[key][0],  title, np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
        results.append(["Support Vector Machine Regression",full_list[key][0],  title, np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
        results.append(["Neural Network Regression", full_list[key][0], title, np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
        results.append(["Elastic Network Regression",full_list[key][0], title, np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 
    except:
            pass

In [358]:
#BLOSUM 62
blosum.update(((b,a),val) for (a,b),val in list(blosum.items()))

def score_pairwise(seq1, seq2, matrix, gap_s, gap_e, gap = True):
    for A,B in zip(seq1, seq2):
        diag = ('-'==A) or ('-'==B)
        yield (gap_e if gap else gap_s) if diag else matrix[(A,B)]
        gap = diag

In [359]:
a = dframe['AgrobacteriumTumefaciens5A']
b = dframe['AgrobacteriumTumefaciens5AW127']

In [360]:
print(sum(score_pairwise(a, b, blosum, -5, -1)))

1521


In [363]:
n

159

In [361]:
n = len(dframe.columns)
enc_seq = np.zeros((n,n))

In [364]:
i = 0

for a in dframe.columns:
    j = 0
    for b in dframe.columns:
        enc_seq[i,j] = sum(score_pairwise(dframe[a], dframe[b], blosum, -5, -1))
        j += 1
    i += 1

KeyError: ('AgrobacteriumTumefaciens5AL178', 'AgrobacteriumTumefaciens5AL178')

In [365]:
dframe[a]

,AgrobacteriumTumefaciens5AL178,AgrobacteriumTumefaciens5AL178
0,-,-
1,-,-
2,-,-
3,-,-
4,-,-
...,...,...
1292,-,-
1293,-,-
1294,-,-
1295,-,-


In [263]:
set(list(dframe[a]))

{'-',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y'}

In [366]:
dframe[b]

,AgrobacteriumTumefaciens5AL178,AgrobacteriumTumefaciens5AL178
0,-,-
1,-,-
2,-,-
3,-,-
4,-,-
...,...,...
1292,-,-
1293,-,-
1294,-,-
1295,-,-


In [264]:
set(list(dframe[b]))

{'-',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y'}